# Laboratorium 1 - A* vs Dijkstra / Tabu search
### Lukasz Fabia 272724


### Jak zacząć?

Python3, najlepiej wersja > 3.11.

```bash
python -m venv .venv
source .venv/bin/activate
pip install -r requirements.txt
```

## Uwaga 

Ze względu na to, że kompliowałem `ipynb` do `pdf` to wpyłnąć na wyświetlanie niektórych danych w szczególności tabel. 

## Teoria i cel

Celem tego zdania jest znalezienie najkrótszego (min. droga) przejazdu MPK lub przejechanie z punktu A do B w najkrótszym czasie (min. czasu).

Do tego problemu optymalizacji należy wykorzystać dwa popularne algorytmy do wyszukiwania najktrótszych ścieżek - A* i Dijkstra. 

---

### Dijkstra

**Algorytm zachłanny**, znajdujący najkrótszą ścieżkę od węzła startowego do wszystkich innych węzłów w grafie. Wykorzstuje wagi (u mnie i w zadaniu korzysta on z kosztu czasu).

### A*

Jest sprowadzalny do **Dijkstry**. Można powiedzieć, że jest taką ulepszoną wersją i jest częściej używany ze względu na optymalność. A* korzysta z heurystyk, żeby oszacować, czy opłaca się wybierać taka a nie inną ścieżkę. Warto dodać, że działa to gdy wiemy czego szukamy, czyli znany jest _target_.

## Jak zbudowałem strukturę grafu?

Generalnie w `/models/graph.py` mamy całą strukturę składa się ona z `Node`, `Edge` i coś co agreguje w sobie węzły, czyli graf. W klasie grafu znajduje się tylko słownik [ulica: węzeł]. 

Kolejnym krokiem było parsowanie danych i wrzucenie ich do grafu. Iterowałem po wierszach i:

1. Wyciągałem dane z wiersza i robiłem z tego krawędź.

2. Dodawałem startowy i końcowy przystanek do słownika. Dodawanie działa w taki sposób, że jeśli ulicy nie ma w kluczach to dodaje dane tej ulicy pod tym kluczem.

3. Aktualizacja krawędzi, czyli do listy startowego węzła dodaje nowy edge z podpiętymi węzłami.


W ten sposób, wystarczy utworzyć obiekt i otrzymujemy `Graf skierowany ważony`. Gdzie **waga** to czas przejazdu w minutach z A do B, a **skierowany** dlatego, że dodaje nową krawędź do startowego node'a. Więcej na temat struktury w kodzie źródłowym [models/graph.py](models/graph.py).

Ostatnia uwaga, przyjąłem, że przy szukaniu połączenia (krawędzi) mogę czekać maksymalnie **15 minut** (czyli od teraz + 15 minut), aby ograniczyć liczbę bezsensownych połączeń. Przykład: jestem na Grunwaldzie, a algorytm proponuje mi połączenie, które odjeżdża za godzinę, ale podróż trwa tylko **12 minut do celu**. Taka optymalizacja.

## Strategie działania

Ten problem można fajnie rozwiązać za pomoca strategii. Logika leży tylko w wybieraniu najbardziej **optymalnej** ścieżki. Zatem zbudowałem sobie szkielet z częścią wspólną tych algorytmów i będę manipulować tylko w konkretnych implementacjach wyborem ścieżki. W ten sposób będzie można tworzyć łatwiej nowe odmiany tych algorytmów.

**Odpowiedź** na konieć to jest czas w formacie `HH:MM:SS`, żeby było czytelniej, _liczba odwiedzonych wierzchołków_ i _czas wykonania_(w ms).

### Zadanie 1a 

Algorytm wyszukiwania najkrótszej ścieżki z A do B za pomocą algorytmu Dijkstry w oparciu o kryterium czasu.

### Inicjalizacja grafu

In [1]:
from parser import to_graph
from dijkstra import Dijkstra
from a import AStarMinTime, AStarMinTransfers, AStarModified
from datetime import time


g = to_graph()

In [2]:
# Times
t1 = time(hour=8, minute=18, second=0)
t2 = time(hour=20, minute=50, second=0)
t3 = time(hour=7, minute=20, second=0)

In [3]:
d_engine = Dijkstra(g)

_ = d_engine.search("Muchobór Wielki", "Mroźna", t1)
_ = d_engine.search("Zajezdnia Obornicka", "Bałtycka", t2)
_ = d_engine.search("Wyszyńskiego", "PL. GRUNWALDZKI", t3)

It took: 778.70 ms

Results for Dijkstra:
Line  Departure   Start Stop     Arrival     End Stop       Cost  
----------------------------------------------------------------------
119   08:21:00    Muchobór Wie   08:22:00    Stanisławows    4     
119   08:22:00    Stanisławows   08:23:00    Trawowa         5     
119   08:23:00    Trawowa        08:24:00    Krzemienieck    6     
119   08:24:00    Krzemienieck   08:25:00    Końcowa         7     
119   08:25:00    Końcowa        08:27:00    Ostrowskiego    9     
119   08:27:00    Ostrowskiego   08:28:00    FAT             10    
5     08:28:00    FAT            08:29:00    Hutmen          11    
5     08:29:00    Hutmen         08:30:00    Bzowa (Centr    12    
5     08:30:00    Bzowa (Centr   08:31:00    pl. Srebrny     13    
5     08:31:00    pl. Srebrny    08:32:00    Stalowa         14    
5     08:32:00    Stalowa        08:34:00    Pereca          16    
5     08:34:00    Pereca         08:35:00    Grabiszyńska    17    
5   

#### Wnioski

1. Minimalny czas podróży
    
    - gwarantuje nam, że koszt faktycznie będzie najmniejszy w moim przypadku - czas podróży, ale gdy warunki będą idealne tj. brak opóźnień tramwajów.

    - można pomyśleć nad algorytmem, który obsłuży sytuacje losowe.

2. Liczba odwiedzonych wierzchołków

    - jest ona całkiem spora, ponieważ przeszukuje wszystko. 

3. Performance

    - długi czas wykonania się dla tras gdzie mamy kilka przystanków to kilkanaście ms, ale liczba rośnie w przypadku trudniejszych odcinków. Jest to spowodowane przeszukiwaniem wszerz co też nijako wiąże się z dużą ilością odwiedzonych wierzchołków. 

    - sam język nie jest demonem prędkości, można przepisać na coś szybszego jak (np. Rust, C/C++).

4. Optymalizacje

    - użycie algorytmu z heurystyką np. A*


### Zadanie 1b 
Algorytm wyszukiwania najkrótszej ścieżki z A do B za pomocą algorytmu A* w oparciu o kryterium czasu.

Jako, że A* to jest modyfikacja Dijkstry to po prostu wartość `priority` to będzie strategia obliczania kosztu dla mojej klasy w tym wypadku klasycznego A*. Poniżej snipped kodu z [a.py](a.py) z liczeniem priorytetu.

```python
priority = self.cost_strategy(
              new_cost=new_cost,
              end_node=end_node,
              next_end_node=next_edge.end_node,
          )
```

A tutaj heurystyka, zastosowana w szukaniu obiecującej ścieżki. Akturat do policzenia odległość użyłem biblioteki `geopy` ze względu na współrzędne geograficzne. Wszystkie heurystyki znajdują się w [search.py](search.py).

```python
def cost_strategy(self, new_cost, **kwargs):
    return new_cost + self._geo_heuristic(
        a=kwargs["end_node"], b=kwargs["next_end_node"]
    )
```


In [4]:
a_engine = AStarMinTime(g)
_ = a_engine.search("Muchobór Wielki", "Mroźna", t1)
_ = a_engine.search("Zajezdnia Obornicka", "Bałtycka", t2)
_ = a_engine.search("Wyszyńskiego", "PL. GRUNWALDZKI", t3)

It took: 62.93 ms

Results for AStarMinTime:
Line  Departure   Start Stop     Arrival     End Stop       Cost  
----------------------------------------------------------------------
119   08:21:00    Muchobór Wie   08:22:00    Stanisławows    4     
119   08:22:00    Stanisławows   08:23:00    Trawowa         5     
119   08:23:00    Trawowa        08:24:00    Krzemienieck    6     
119   08:24:00    Krzemienieck   08:25:00    Końcowa         7     
119   08:25:00    Końcowa        08:27:00    Ostrowskiego    9     
119   08:27:00    Ostrowskiego   08:28:00    FAT             10    
5     08:28:00    FAT            08:29:00    Hutmen          11    
5     08:29:00    Hutmen         08:30:00    Bzowa (Centr    12    
5     08:30:00    Bzowa (Centr   08:31:00    pl. Srebrny     13    
5     08:31:00    pl. Srebrny    08:32:00    Stalowa         14    
5     08:32:00    Stalowa        08:34:00    Pereca          16    
5     08:34:00    Pereca         08:35:00    Grabiszyńska    17    
5

#### Wnioski

1. Minimalny czas podróży

    - rozwiązanie jest gorsze jak można było się spodziewać, ale bardzo szybko je dostajemy

    - warto dodać, że jest to zależne od heurystyki
    

2. Liczba odwiedzonych wierzchołków

    - liczba jest o wiele mniejsza od rozwiązania problemu za pomocą **Dijkstry**

    - jest to spowodowane użyciem heurystyki, która przeszukuje rokujące węzły

    - całkiem mało odwiedzonych, ale wynika to, że szuka tylko tam gdzie jest obiecująco. 


3. Performance

    - tu jest znaczenie lepiej bo nie przeszukujemy wszystkiego

4. Opytmalizacje

    - można dodać heurystyke, która by sprawdzała kierunek, którym ma się kierować na podstawie kątów obliczanych z koordynatów punktu A i B.   

#### Zadanie 1c

**Minimalizacja liczby przesiadek**. Intuicja podpowiada, że trzeba będzie wprowadzić coś w stylu kary, dodatkowego kosztu doliczanego do ścieżki, która może i jest najkrótszą, ale wymaga przesiadki właśnie. W ten sposób algorytm będzie brał pod uwagę ścieżki, które nie wymagają zmiany `line`. Tak naprawdę cała zabawa sprowadziła się do dodatania do nowego kosztu sprawdzienie ile kosztować przesiadka. Poniżej _snipped_ z source kodu, który znajduje się [a.py](a.py). 

```python
new_cost = (
            cost_so_far[current_node]
            + self.graph.compute_cost(next_edge, current_time)
            + self.graph.line_change_cost(
                edge=came_from[current_node.name], next_edge=next_edge
            )
        )
```


In [5]:

a_mut_engine = AStarMinTransfers(g)

_ = a_mut_engine.search("Muchobór Wielki", "Mroźna", t1)
_ = a_mut_engine.search("Zajezdnia Obornicka", "Bałtycka", t2)
_ = a_mut_engine.search("Wyszyńskiego", "PL. GRUNWALDZKI", t3)

It took: 174.46 ms

Results for AStarMinTransfers:
Line  Departure   Start Stop     Arrival     End Stop       Cost  
----------------------------------------------------------------------
119   08:21:00    Muchobór Wie   08:22:00    Stanisławows    4     
119   08:22:00    Stanisławows   08:23:00    Trawowa         5     
119   08:23:00    Trawowa        08:24:00    Krzemienieck    6     
119   08:24:00    Krzemienieck   08:25:00    Końcowa         7     
119   08:25:00    Końcowa        08:27:00    Ostrowskiego    9     
119   08:27:00    Ostrowskiego   08:28:00    FAT             10    
134   08:37:00    FAT            08:38:00    Hutmen          70    
134   08:38:00    Hutmen         08:40:00    Bzowa (Centr    72    
134   08:40:00    Bzowa (Centr   08:41:00    pl. Srebrny     73    
134   08:41:00    pl. Srebrny    08:43:00    Stalowa         75    
126   08:52:00    Stalowa        08:53:00    Pereca          135   
126   08:53:00    Pereca         08:55:00    Grabiszyńska    13

# Zadanie 1d - Astar Modified

Modyfikacja algorytmu gdzie min. przesiadki, porównanie z 1c.

Skupię się bardziej na pierwszej trasie bo jest więcej liczenia, ale małe trasy tez przeanalizuje, ale krócej.

A* zopytmalizowany dał nam **10** przesiadek na pierwszej trasie, obliczył to w czasie **~122 ms** i podróz potrwa **1h 47min**.

A* wersja z 1c podstawowa dała 


In [6]:
mod = AStarModified(g)
_ = mod.search_optimized("Muchobór Wielki", "Mroźna", t1)
_ = mod.search_optimized("Zajezdnia Obornicka", "Bałtycka", t2)
_ = mod.search_optimized("Wyszyńskiego", "PL. GRUNWALDZKI", t3)

It took: 59.22 ms

Results for AStarModified:
Line  Departure   Start Stop     Arrival     End Stop       Cost  
----------------------------------------------------------------------
119   08:21:00    Muchobór Wie   08:22:00    Stanisławows    4     
119   08:22:00    Stanisławows   08:23:00    Trawowa         5     
119   08:23:00    Trawowa        08:24:00    Krzemienieck    6     
119   08:24:00    Krzemienieck   08:25:00    Końcowa         7     
119   08:25:00    Końcowa        08:27:00    Ostrowskiego    9     
119   08:27:00    Ostrowskiego   08:28:00    FAT             10    
5     08:28:00    FAT            08:29:00    Hutmen          11    
5     08:29:00    Hutmen         08:30:00    Bzowa (Centr    12    
5     08:30:00    Bzowa (Centr   08:31:00    pl. Srebrny     13    
5     08:31:00    pl. Srebrny    08:32:00    Stalowa         14    
5     08:32:00    Stalowa        08:34:00    Pereca          16    
5     08:34:00    Pereca         08:35:00    Grabiszyńska    17    


#### Wnioski

Wynik jest podobny do zwykłego A*, gdzie minimalizowany był czas. Tutaj mieliśmy minimalizować przesiadki i powiedzmy, że się udało na przykładowych trasach. Mamy mniej zmian linii, czas nieznaczenie się wydłużył i widać doliczoną karę za przesiadkę. W niektórych testach liczba odwiedzonych wierzchołków jest mniejsza. 

# Podsumowanie pierwszej części listy

Generalnie zgodnie z założeniem Dijkstra zwraca najlepsze wyniki w nie najlepszym czasie. Udało się w miarę sensownie zaimplementować A* (minimalizacja czasu albo przystanków), który zwaraca dobrą odpowiedź w naprawdę fajnym czasie. Udało się napisać względnie bez duplikacji kodu przez co łatwo można pisać swoje implementacje, które jakoś optymalizują wybór ścieżki. 

Największy problem jednak sprawił sam `Python`, jako osoba, która raczej jest przyzwyczajona do **Go**, **TypeScripta**, **Javy** no to było ciężko debugować, ale trzeba przyznać, że _JupyterNotebook_ pomógł w procesie. Last but not least, ułatwieniem było, że doba nie trwała 24h tylko więcej to pomogło się skupiać na policzeniu kosztu w minutach między węzłami (przystankami).    




## Tabu Search

**Metaheurystyka** polegająca na interacyjnym przeszukiwaniu sąsiedztwa, uwzględnia zestaw ruchów niedozwolonych. Wymaga dodakowej pamięci w porównaniu do **przeszukiwania lokalnego**.

W zadaniu należy przechejać przez wszystkie przystanki i wrócić się do punktu początkowego po przez minimalizację:

- czasu

- przesiadek

Podobnie jak w poprzednim zadaniu tutaj też postarałem się aby nie duplikować kodu i poprostu parametry w zadaniu będę przyjmować opcjonalnie.

W implementacji `wycieczki` po mieście użyłem A*, ponieważ Dijkstra wykonywałby się znacznie dłużej.



# Opis rozwiązania

Co do samej implementacji mojego Tabu zrobiłem strategie, gdzie przyjmuje argumenty takie jak:

- dłogosc tablicy (u mnie jest 10)

- czy ma uzyc apiracji

- liczba probek

- dodatkowa informacja nt. tego ile mozna iterowac kiedy nie ma zmian, czyli to jest cos w stylu mam if sprawdzajacego czy znaleziono najlepszego sąsiada po względem kosztu no i jak się nie znalazło to się inkrementuje a potem jak przekroczę granicę `max_no_imporove` to generuje nowe rowiązanie. 

## Przykład użycia dla min. czasu

In [7]:
from tabu import Tabu
max_iter = 50
points = ["PL. GRUNWALDZKI", "Dubois", "DWORZEC GŁÓWNY"]
src = "Wyszyńskiego"
t = Tabu(g=g, t=t2, points=points, src=src, max_iter=max_iter)

# Tabu Search standardowy (a)

In [8]:
best_sln = t.search()
t.set_points(best_sln)
t.go_on_a_trip()


It took: 535.97 ms
It took: 0.01 ms
It took: 5.34 ms

Results for AStarModified:
Line  Departure   Start Stop     Arrival     End Stop       Cost  
----------------------------------------------------------------------
A     20:50:00    Wyszyńskiego   20:52:00    Ogród Botani    2     
A     20:52:00    Ogród Botani   20:53:00    Katedra         3     
A     20:53:00    Katedra        20:54:00    Urząd Wojewó    4     
A     20:54:00    Urząd Wojewó   20:55:00    Poczta Główn    5     
A     20:55:00    Poczta Główn   20:57:00    skwer Krasiń    7     
145   21:04:00    skwer Krasiń   21:06:00    DWORZEC GŁÓW    16    
----------------------------------------------------------------------

Total time (AStarModified): 00:16:00
Total visited nodes: 178

It took: 8.60 ms

Results for AStarModified:
Line  Departure   Start Stop     Arrival     End Stop       Cost  
----------------------------------------------------------------------
K     21:07:00    DWORZEC GŁÓW   21:11:00    GALERIA DO

### Tabu Search dobór długości tablicy T (b)

Dostosowuje długość listy tabu w trakcie działania, w zależności od postępu algorytmu. Długość T to 10, ale można zmienić. 

In [9]:
best_sln = t.dynamic_search()
t.set_points(best_sln)
t.go_on_a_trip()


It took: 1147.07 ms
It took: 6.03 ms

Results for AStarModified:
Line  Departure   Start Stop     Arrival     End Stop       Cost  
----------------------------------------------------------------------
1     20:53:00    Wyszyńskiego   20:55:00    Nowowiejska     5     
1     20:55:00    Nowowiejska    20:56:00    Słowiańska      6     
1     20:56:00    Słowiańska     20:58:00    DWORZEC NADO    8     
14    21:02:00    DWORZEC NADO   21:03:00    Paulińska       13    
14    21:03:00    Paulińska      21:05:00    Dubois          15    
----------------------------------------------------------------------

Total time (AStarModified): 00:15:00
Total visited nodes: 230

It took: 6.65 ms

Results for AStarModified:
Line  Departure   Start Stop     Arrival     End Stop       Cost  
----------------------------------------------------------------------
13    21:13:00    Dubois         21:15:00    pl. Bema        10    
13    21:15:00    pl. Bema       21:16:00    Na Szańcach     11    
13 

### Tabu Search aspiracja (c)

Pozwala zaakceptować rozwiązania z listy tabu, jeśli są lepsze niż poprzednie najlepsze rozwiązanie.

In [10]:
best_sln = t.aspiration_search()
t.set_points(best_sln)
t.go_on_a_trip()


It took: 2116.23 ms
It took: 0.01 ms
It took: 0.00 ms
It took: 0.00 ms
It took: 2.43 ms

Results for AStarModified:
Line  Departure   Start Stop     Arrival     End Stop       Cost  
----------------------------------------------------------------------
1     20:56:00    Wyszyńskiego   20:57:00    Prusa           7     
1     20:57:00    Prusa          20:59:00    Piastowska      9     
1     20:59:00    Piastowska     21:02:00    PL. GRUNWALD    12    
----------------------------------------------------------------------

Total time (AStarModified): 00:12:00
Total visited nodes: 76

It took: 4.95 ms

Results for AStarModified:
Line  Departure   Start Stop     Arrival     End Stop       Cost  
----------------------------------------------------------------------
13    21:02:00    PL. GRUNWALD   21:03:00    most Grunwal    1     
146   21:15:00    most Grunwal   21:17:00    Poczta Główn    15    
146   21:17:00    Poczta Główn   21:19:00    skwer Krasiń    17    
146   21:19:00    skw

### Tabu Search sampling (d) 

Wybieram losowo k list z sąsiadami, w celu zwiększenia różnorodności poszukiwań


In [12]:
best_sln = t.sampling_search(sample=10)
t.set_points(best_sln)
t.go_on_a_trip()


It took: 2911.25 ms
It took: 0.01 ms
It took: 0.00 ms
It took: 0.00 ms
It took: 0.00 ms
It took: 5.55 ms

Results for AStarModified:
Line  Departure   Start Stop     Arrival     End Stop       Cost  
----------------------------------------------------------------------
A     20:50:00    Wyszyńskiego   20:52:00    Ogród Botani    2     
A     20:52:00    Ogród Botani   20:53:00    Katedra         3     
A     20:53:00    Katedra        20:54:00    Urząd Wojewó    4     
A     20:54:00    Urząd Wojewó   20:55:00    Poczta Główn    5     
A     20:55:00    Poczta Główn   20:57:00    skwer Krasiń    7     
145   21:04:00    skwer Krasiń   21:06:00    DWORZEC GŁÓW    16    
----------------------------------------------------------------------

Total time (AStarModified): 00:16:00
Total visited nodes: 178

It took: 19.21 ms

Results for AStarModified:
Line  Departure   Start Stop     Arrival     End Stop       Cost  
----------------------------------------------------------------------
K 